In [150]:
%%capture
# Compile and import local pyrossgeo module
import os, sys
owd = os.getcwd()
os.chdir('../../')
sys.path.insert(0,'../../')
!python setup.py build_ext --inplace
os.chdir(owd)

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyrossgeo

import pandas as pd
import json

In [140]:
cn = pd.read_csv("london_simulation/commuter_networks.csv")

# Fix column names

cn = cn.rename(columns = {'N_move' : '# to move'})
cn.insert(5, "% to move", -1)

# Fix that 100% return home

cn.loc[ cn['Home'] == cn['To'], '# to move'] = -1
cn.loc[ cn['Home'] == cn['To'], '% to move'] = 1.0

cn.head()

# Remove commutes to same node

cn = cn[ cn['From'] != cn['To'] ]

cn = cn.reset_index()

In [141]:
allow_class = [
    ('S', True),
    ('E', True),
    ('A', True),
    ('Ia1', True),
    ('Ia2', True),
    ('Ia3', True),
    ('Is1', True),
    ('Is2', False),
    ('Is3', False),
    ('R', True),
]

# Drop the current allow_O columns
cn = cn.iloc[:,:10]

# Set allow settings
for O, allow_O in allow_class:
    cn[ "Allow %s" % O ] = 1 if allow_O else 0
    
# Allow people to return home
cn.loc[ cn['Home'] == cn['To'],"Allow %s" % allow_class[0][0]:] = 1

In [143]:
min_num_moving = 20

delete_rows = []

for i, row in cn.loc[ cn['Home'] == cn['From'] ].iterrows():
    if row['# to move'] < min_num_moving:
        delete_rows.append(i)
        delete_rows.append(i+1) # Delete the returning commuting edge as well
        
cn = cn.drop(delete_rows)